In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import math

In [2]:
def random_start_indices(start, end, num_samples):
    return np.random.randint(start, end, num_samples)

In [3]:
i_3 = random_start_indices(start=32558400,end=118958399, num_samples =1500)
i_4 = random_start_indices(start=0, end=35360700, num_samples=1500)
i_6 = random_start_indices(start=57993700,end=114153699, num_samples =1500)

In [4]:
def get_selected_rows(parquet_path, indices, csv_path, chunk_size):
    df = pd.read_parquet(parquet_path)
    rows= []
    for index in indices:
        for i in range(chunk_size):
            rows.append(df.loc[index+i*100])

    df = pd.DataFrame(rows)
    df.to_csv(csv_path)

In [54]:
get_selected_rows(parquet_path="./cleaned_parquet/007/Month 2/DD072024007_left wrist_101697_2024-09-18 11-15-01.parquet", indices=i_2, csv_path="month_2.csv", chunk_size=10)

In [5]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 3/DD072024007_left wrist_101696_2024-10-22 15-22-36.parquet", indices=i_3, csv_path="month_3.csv", chunk_size=10)

In [6]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 4/DD072024007_left wrist_101697_2024-11-20 13-12-43.parquet", indices=i_4, csv_path="month_4.csv", chunk_size=10)

In [57]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 5/DD072024007_left wrist_101699_2024-11-19 10-32-50.parquet", indices=i_5, csv_path="month_5.csv", chunk_size=10)

In [7]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 6/DD072024007_left wrist_101696_2025-01-30 13-12-54.parquet", indices=i_6, csv_path="month_6.csv", chunk_size=10)

In [59]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 7/DD072024007__100889_2025-03-26 12-05-58.parquet", indices=i_7, csv_path="month_7.csv", chunk_size=10)

In [12]:
def labeling_a_seq(seq_start_date, seq_end_date, df_labels):
    df_labels["start_date"] = pd.to_datetime(df_labels["start_date"])
    df_labels["end_date"] = pd.to_datetime(df_labels["end_date"])
    for i in range(len(df_labels["average"])):
        if df_labels["start_date"].iloc[i] < seq_start_date and df_labels["end_date"].iloc[i] > seq_end_date:
            return df_labels["average"].iloc[i]
    

In [13]:
df_labels = pd.read_csv("cleaned_parquet/007/labels.csv")
seq_start_date= pd.to_datetime("2024-10-07")
seq_end_date=pd.to_datetime("2024-10-07")

labeling_a_seq(seq_start_date, seq_end_date, df_labels)

'neutral'

In [14]:
def label_seqs(csv_path, seq_start_date, seq_end_date):
    df = pd.read_csv(csv_path)
    df["date"] = pd.to_datetime(df["date"])
    df_labels = pd.read_csv("cleaned_parquet/007/labels.csv")
    labels=[]

    for i in range(0, len(df["date"]), 10):
        label = labeling_a_seq(seq_start_date, seq_end_date, df_labels)
        for j in range(10):
            labels.append(label)

    df["label"] = labels
    return df

In [18]:
def label(csv_path, l):
    df = pd.read_csv(csv_path)
    labels = np.full(15000,l)
    df["label"] = labels
    return df

In [19]:
df_3 = label(csv_path="cleaned_parquet/007/month_3.csv", l = "neutral")
df_4 = label(csv_path="cleaned_parquet/007/month_4.csv", l = "bad")
df_6 = label(csv_path="cleaned_parquet/007/month_6.csv", l = "good")

df_all = pd.concat([df_3, df_4, df_6])
df_all.to_csv("cleaned_parquet/007/sample.csv")